## Utils

In [ ]:
def pprint(text):
    import textwrap
    wrapped_text = textwrap.fill(text, width=100) 
    print(wrapped_text)

## LLMs

In [36]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

emb_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/Users/maksim.rostov/pdev/freestyling/agents/hf-course/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from llama_index.llms.ollama import Ollama  

chat_model = Ollama(model="qwen2:7b")

## Researcher: An Agent Workflow

 1) take a topic to write a blogpost about and subdivide it into sections
Let's create an agent workflow that would: 
 2) The first section should define the variables, definitions, concepts and terminology that are going to be used for explaining (mathematicaly) the problem   
    - Briefly explain those concepts if they are complicated 
 3) Look up websites that explain the topic  
    - Download the selected websites and process their data (make a note that you need to refence them)
    - Select websites that best fit the subsections identified before  
    - Decide which website is the best at explaining the topic based on how it covers the subsections
4) Analyse the website and change rearrange its content to fit the desired subsection structure and the variables and defintions  
     

In [ ]:
from llama_index.core.agent.workflow import ReActAgent, FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context

import mlflow

mlflow.set_experiment(experiment_name="Test")
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.llama_index.autolog()


def get_relevant_webpages(query: str) -> list:
    """
    Gets the relevant webpages urls for a query.

    Args:
        query (str): what to search online on www.

    Returns:
        list: list of url links (websites) related to the query 
    """
    search_ggg = DDGS()
    results = search_ggg.text(query)
    return results

web_search_tool = FunctionTool.from_defaults(
    fn=get_relevant_webpages,
    name="get_relevant_webpages",
    description="Useful for getting a list of relevant webpages (url links) for a particular query. " \
    "Together with the web links the list also include a short information about the answer for the query."
)

search_agent = ReActAgent(
    name = "Web Searcher", 
    description = "Search the web give links to the relevant pages it found",
    system_prompt = "",
    tools=[web_search_tool],
    verbose=True,
    llm=chat_model,
)

ctx = Context(search_agent)
answer = await search_agent.run("History of Prussia, search online with the web search tool", ctx=ctx)
print(answer)
pprint(answer.response.blocks[0].text)